# Day 25: Sea Cucumber

[*Advent of Code 2021 day 25*](https://adventofcode.com/2021/day/25) and [*solution megathread*](https://redd.it/ro2uav)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/25/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F25%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys

sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

## Comments

Hmm... an iterative, chaotic problem, not a lot of room for smarts, largely a matter of making iterations fast enough?

Funny - in `destination`, I didn't have an [off-by-one error](https://en.wikipedia.org/wiki/Off-by-one_error) as one might expect, but a [modulus-precedence/associativity error](https://en.wikipedia.org/wiki/Order_of_operations)!

In [5]:
testdata = []
testdata.append(("""v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>""".splitlines(), 58))

inputdata = downloaded['input'].splitlines()

In [6]:
print(f'{len(inputdata[0])} x {len(inputdata)}')

139 x 137


In [7]:
from itertools import count

Coord = tuple[int, int]


def parse_data(data: list[str]) -> tuple[set[Coord], set[Coord], Coord]:
    east, south = set(), set()
    size = (len(data[0]), len(data))
    for j, row in enumerate(data):
        for i, slot in enumerate(row):
            match slot:
                case '>':
                    east.add((i, j))
                case 'v':
                    south.add((i, j))
    return east, south, size


def show(east: set[Coord], south: set[Coord], size):
    result = []
    for j in range(size[1]):
        row = []
        for i in range(size[0]):
            if (i, j) in east:
                row.append('>')
            elif (i, j) in south:
                row.append('v')
            else:
                row.append('.')
        result.append(''.join(row))
    return '\n'.join(result)


def destination(size: Coord, pos: Coord, east_phase: bool) -> Coord:
    if east_phase:
        return (pos[0] + 1) % size[0], pos[1]
    else:
        return pos[0], (pos[1] + 1) % size[1]


def iterate_phase(occupied: set[Coord],
                  size: Coord,
                  cucumbers: set[Coord],
                  east_phase: bool,
                  debug: bool = False) -> tuple[bool, set[Coord]]:
    next_cucumbers = set()
    movement = False
    for c in cucumbers:
        dest = destination(size, c, east_phase)
        if dest not in occupied:
            movement = True
            next_cucumbers.add(dest)
        else:
            next_cucumbers.add(c)
    return movement, next_cucumbers


def iterate(east: set[Coord],
            south: set[Coord],
            size: Coord,
            debug: bool = False) -> tuple[bool, set[Coord], set[Coord]]:
    occupied = east.union(south)
    movement_east, next_east = iterate_phase(occupied,
                                             size,
                                             east,
                                             True,
                                             debug=debug)
    occupied = next_east.union(south)
    movement_south, next_south = iterate_phase(occupied,
                                               size,
                                               south,
                                               False,
                                               debug=debug)
    return movement_east or movement_south, next_east, next_south


def my_part1_solution(data: list[str],
                      debug: bool = False) -> int:
    east, south, size = parse_data(data)
    if debug:
        print('Initial state:')
        print(show(east, south, size))
    for i in count(start=1):
        movement, east, south = iterate(east,
                                        south,
                                        size,
                                        debug=debug)
        if debug:
            print(f'\nAfter {i} steps:')
            print(show(east, south, size))
        if not movement:
            return i

In [8]:
assert(my_part1_solution(testdata[0][0], debug=True) == testdata[0][1])

Initial state:
v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>

After 1 steps:
....>.>v.>
v.v>.>v.v.
>v>>..>v..
>>v>v>.>.v
.>v.v...v.
v>>.>vvv..
..v...>>..
vv...>>vv.
>.v.v..v.v

After 2 steps:
>.v.v>>..v
v.v.>>vv..
>v>.>.>.v.
>>v>v.>v>.
.>..v....v
.>v>>.v.v.
v....v>v>.
.vv..>>v..
v>.....vv.

After 3 steps:
v>v.v>.>v.
v...>>.v.v
>vv>.>v>..
>>v>v.>.v>
..>....v..
.>.>v>v..v
..v..v>vv>
v.v..>>v..
.v>....v..

After 4 steps:
v>..v.>>..
v.v.>.>.v.
>vv.>>.v>v
>>.>..v>.>
..v>v...v.
..>>.>vv..
>.v.vv>v.v
.....>>vv.
vvv>...v..

After 5 steps:
vv>...>v>.
v.v.v>.>v.
>.v.>.>.>v
>v>.>..v>>
..v>v.v...
..>.>>vvv.
.>...v>v..
..v.v>>v.v
v.v.>...v.

After 6 steps:
v.v>..>vv>
.vv...>>v.
v>..v>.>>v
>.v>.>..>>
.v.>v.vv..
..v>>>vv..
..>..v>vv.
..v..>>...
v...v>.v.v

After 7 steps:
..v.>.>vv>
v.v...>>v.
.v>.v.>>>v
v>..>.>.>>
..v>v.vv..
.v.>>>vv..
..v>.v>v..
.....>.>v.
v.v.v.>v.v

After 8 steps:
>.v.v>>vvv
......>>v.
v.v>..>>>v
.v>.v>.>>>
v..>v.vv..
..v>>>vv..

In [9]:
my_part1_solution(inputdata)

582

In [10]:
HTML(downloaded['part1_footer'])

## Part Two

In [11]:
HTML(downloaded['part2'])

In [12]:
HTML(downloaded['part2_footer'])